In [1]:
import time

from sklearn.datasets import load_iris, fetch_20newsgroups, fetch_california_housing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

In [17]:
# 1.完成对facebook数据的k近邻的机器学习，并通过网格搜索找到最佳的kneighbors
data = pd.read_csv("./data/FBlocation/train.csv")#读取数据
print(data.head(10))#查看数据的前10行
print(data.shape)#查看数据的维度
print(data.info())#查看数据的类型
data = data.query("x > 1.0 &  x < 1.25 & y > 2.5 & y < 2.75")#筛选数据)

   row_id       x       y  accuracy    time    place_id
0       0  0.7941  9.0809        54  470702  8523065625
1       1  5.9567  4.7968        13  186555  1757726713
2       2  8.3078  7.0407        74  322648  1137537235
3       3  7.3665  2.5165        65  704587  6567393236
4       4  4.0961  1.1307        31  472130  7440663949
5       5  3.8099  1.9586        75  178065  6289802927
6       6  6.3336  4.3720        13  666829  9931249544
7       7  5.7409  6.7697        85  369002  5662813655
8       8  4.3114  6.9410         3  166384  8471780938
9       9  6.3414  0.0758        65  400060  1253803156
(29118021, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29118021 entries, 0 to 29118020
Data columns (total 6 columns):
 #   Column    Dtype  
---  ------    -----  
 0   row_id    int64  
 1   x         float64
 2   y         float64
 3   accuracy  int64  
 4   time      int64  
 5   place_id  int64  
dtypes: float64(2), int64(4)
memory usage: 1.3 GB
None


In [18]:
data.shape

(17710, 6)

In [19]:
data.describe()#查看数据的统计信息

,row_id,x,y,accuracy,time,place_id
count,1.771000e+04,17710.000000,17710.000000,17710.000000,17710.000000,1.771000e+04
mean,1.450569e+07,1.122538,2.632309,82.482101,397551.263128,5.129895e+09
std,8.353805e+06,0.077086,0.070144,113.613227,234601.097883,2.357399e+09
min,6.000000e+02,1.000100,2.500100,1.000000,119.000000,1.012024e+09
25%,7.327816e+06,1.049200,2.573800,25.000000,174069.750000,3.312464e+09
50%,1.443071e+07,1.123300,2.642300,62.000000,403387.500000,5.261906e+09
75%,2.163463e+07,1.190500,2.687800,75.000000,602111.750000,6.766325e+09
max,2.911215e+07,1.249900,2.749900,1004.000000,786218.000000,9.980711e+09


In [22]:
time_value = pd.to_datetime(data['time'], unit='s')#将时间戳转换为日期格式
print(time_value.head(20))#查看转换后的前20行

600     1970-01-01 18:09:40
957     1970-01-10 02:11:10
4345    1970-01-05 15:08:02
4735    1970-01-06 23:03:03
5580    1970-01-09 11:26:50
6090    1970-01-02 16:25:07
6234    1970-01-04 15:52:57
6350    1970-01-01 10:13:36
7468    1970-01-09 15:26:06
8478    1970-01-08 23:52:02
9357    1970-01-04 16:53:19
12125   1970-01-07 03:55:07
14937   1970-01-06 03:46:38
20660   1970-01-08 03:08:15
20930   1970-01-02 21:31:48
21731   1970-01-07 08:52:19
26584   1970-01-04 15:48:09
27937   1970-01-08 03:51:54
30798   1970-01-01 20:58:30
33184   1970-01-06 15:31:39
Name: time, dtype: datetime64[ns]


In [24]:
time_value = pd.DatetimeIndex(time_value)#将时间戳转换为日期格式
print(time_value[0:20])#查看转换后的前20行

DatetimeIndex(['1970-01-01 18:09:40', '1970-01-10 02:11:10',
               '1970-01-05 15:08:02', '1970-01-06 23:03:03',
               '1970-01-09 11:26:50', '1970-01-02 16:25:07',
               '1970-01-04 15:52:57', '1970-01-01 10:13:36',
               '1970-01-09 15:26:06', '1970-01-08 23:52:02',
               '1970-01-04 16:53:19', '1970-01-07 03:55:07',
               '1970-01-06 03:46:38', '1970-01-08 03:08:15',
               '1970-01-02 21:31:48', '1970-01-07 08:52:19',
               '1970-01-04 15:48:09', '1970-01-08 03:51:54',
               '1970-01-01 20:58:30', '1970-01-06 15:31:39'],
              dtype='datetime64[ns]', name='time', freq=None)


In [25]:
data.shape#查看数据的维度

(17710, 6)

In [27]:
print(type(data))#查看数据的类型
data.insert(data.shape[1], 'day', time_value.day)#在数据中插入新的列，表示日期
data.insert(data.shape[1], 'hour', time_value.hour)#在数据中插入新的列，表示小时
data.insert(data.shape[1], 'weekday', time_value.weekday)#在数据中插入新的列，表示星期几

data = data.drop(['time'], axis=1)#删除时间列
data.head()#查看数据

<class 'pandas.core.frame.DataFrame'>


,row_id,x,y,accuracy,place_id,day,hour,weekday
600,600,1.2214,2.7023,17,6683426742,1,18,3
957,957,1.1832,2.6891,58,6683426742,10,2,5
4345,4345,1.1935,2.6550,11,6889790653,5,15,0
4735,4735,1.1452,2.6074,49,6822359752,6,23,1
5580,5580,1.0089,2.7287,19,1527921905,9,11,4


In [28]:
per = pd.Period('2025-01-10 18:00', 'h')#创建时间对象
per.weekday

4

In [29]:
data.describe()#查看数据的统计信息

,row_id,x,y,accuracy,place_id,day,hour,weekday
count,1.771000e+04,17710.000000,17710.000000,17710.000000,1.771000e+04,17710.000000,17710.000000,17710.000000
mean,1.450569e+07,1.122538,2.632309,82.482101,5.129895e+09,5.101863,11.485545,3.092377
std,8.353805e+06,0.077086,0.070144,113.613227,2.357399e+09,2.709287,6.932195,1.680218
min,6.000000e+02,1.000100,2.500100,1.000000,1.012024e+09,1.000000,0.000000,0.000000
25%,7.327816e+06,1.049200,2.573800,25.000000,3.312464e+09,3.000000,6.000000,2.000000
50%,1.443071e+07,1.123300,2.642300,62.000000,5.261906e+09,5.000000,12.000000,3.000000
75%,2.163463e+07,1.190500,2.687800,75.000000,6.766325e+09,7.000000,17.000000,4.000000
max,2.911215e+07,1.249900,2.749900,1004.000000,9.980711e+09,10.000000,23.000000,6.000000


In [30]:
place_count = data.groupby('place_id').count()#按place_id分组，并计算每个分组的数量   
place_count

,row_id,x,y,accuracy,day,hour,weekday
place_id,,,,,,,
1012023972,1,1,1,1,1,1,1
1057182134,1,1,1,1,1,1,1
1059958036,3,3,3,3,3,3,3
1085266789,1,1,1,1,1,1,1
1097200869,1044,1044,1044,1044,1044,1044,1044
...,...,...,...,...,...,...,...
9904182060,1,1,1,1,1,1,1
9915093501,1,1,1,1,1,1,1
9946198589,1,1,1,1,1,1,1


In [31]:
place_count['x'].describe() #查看place_id的数量统计信息

count     805.000000
mean       22.000000
std        88.955632
min         1.000000
25%         1.000000
50%         2.000000
75%         5.000000
max      1044.000000
Name: x, dtype: float64

In [33]:
tf = place_count[place_count.row_id > 3].reset_index()#筛选出数量大于3的place_id，并重置索引
tf 

,place_id,row_id,x,y,accuracy,day,hour,weekday
0,1097200869,1044,1044,1044,1044,1044,1044,1044
1,1228935308,120,120,120,120,120,120,120
2,1267801529,58,58,58,58,58,58,58
3,1278040507,15,15,15,15,15,15,15
4,1285051622,21,21,21,21,21,21,21
...,...,...,...,...,...,...,...,...
234,9741307878,5,5,5,5,5,5,5
235,9753855529,21,21,21,21,21,21,21
236,9806043737,6,6,6,6,6,6,6
237,9809476069,23,23,23,23,23,23,23


In [34]:
data = data[data['place_id'].isin(tf.place_id)]#筛选出数量大于3的place_id
data.shape#查看数据的维度

(16918, 8)

In [35]:
y = data['place_id']#将place_id作为目标变量
x = data.drop(['place_id'], axis=1)#删除place_id列，并将剩余的列作为特征变量
x = x.drop(['row_id'], axis=1)#删除row_id列
print(x.shape)#查看特征的维度
print(x.columns)#查看特征的列名

(16918, 6)
Index(['x', 'y', 'accuracy', 'day', 'hour', 'weekday'], dtype='object')


In [36]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=1)#将数据集分为训练集和测试集
std = StandardScaler()#创建标准化对象
# # # 对测试集和训练集的特征值进行标准化,服务于knn fit
x_train = std.fit_transform(x_train)#fit_transform会进行均值和方差的计算，服务于knn fit
print(std.mean_)#fit_transform会进行均值和方差的计算，服务于knn fit
print(std.var_)#
x_test = std.transform(x_test) #transform不会进行均值和方差的计算，服务于knn predict
print(std.mean_)
print(std.var_)

[ 1.12295735  2.63237278 81.34938525  5.10064628 11.44293821  3.10135561]
[5.98489138e-03 4.86857391e-03 1.19597480e+04 7.32837915e+00
 4.83742660e+01 2.81838404e+00]
[ 1.12295735  2.63237278 81.34938525  5.10064628 11.44293821  3.10135561]
[5.98489138e-03 4.86857391e-03 1.19597480e+04 7.32837915e+00
 4.83742660e+01 2.81838404e+00]


In [37]:
x_train.shape#查看训练集的维度

(12688, 6)

In [38]:
knn = KNeighborsClassifier(n_neighbors=3)#创建knn对象，设置邻居数为3
knn.fit(x_train, y_train)#训练模型
y_predict = knn.predict(x_test)#预测测试集
print("预测的目标签到位置为：", y_predict[0:10])#查看预测结果
print("预测的准确率:", knn.score(x_test, y_test))#查看准确率

预测的目标签到位置为： [5689129232 1097200869 6097504486 9632980559 6424972551 1097200869
 3952821602 3533177779 1435128522 3312463746]
预测的准确率: 0.46430260047281324


In [39]:
print(y_test[0:10] )#查看真实的目标签到位置

16751286    1893548673
12423167    1097200869
7517023     6097504486
4400015     9632980559
26212472    6424972551
7089828     4022692381
10935607    2327054745
25025511    3533177779
27755137    1435128522
19678934    3312463746
Name: place_id, dtype: int64


In [40]:
param = {"n_neighbors": [3, 5, 10, 12, 15],'weights':['uniform', 'distance']}#设置网格搜索的参数
gc = GridSearchCV(knn, param_grid=param, cv=3)#创建网格搜索对象，设置交叉验证的折数为3
gc.fit(x_train, y_train) #训练模型
print("在测试集上准确率：", gc.score(x_test, y_test))#查看在测试集上的准确率
print("在交叉验证当中最好的结果：", gc.best_score_) #查看在交叉验证当中最好的结果
print("选择最好的模型是：", gc.best_estimator_)#查看选择最好的模型 
print("每个超参数每次交叉验证的结果：")#查看每个超参数每次交叉验证的结果
gc.cv_results_#查看每个超参数每次交叉验证的结果

C:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


在测试集上准确率： 0.49763593380614657
在交叉验证当中最好的结果： 0.4816362349278435
选择最好的模型是： KNeighborsClassifier(n_neighbors=12, weights='distance')
每个超参数每次交叉验证的结果：


{'mean_fit_time': array([0.00919739, 0.00772405, 0.00989294, 0.00819349, 0.00773199,
        0.00768232, 0.00812769, 0.00783205, 0.00749707, 0.0078241 ]),
 'std_fit_time': array([0.00125633, 0.00021748, 0.00262742, 0.00022082, 0.00017639,
        0.00017311, 0.00054448, 0.00016993, 0.0003715 , 0.00019736]),
 'mean_score_time': array([0.15606848, 0.06875285, 0.18893981, 0.07219521, 0.18453209,
        0.08768352, 0.18673666, 0.094757  , 0.19382715, 0.11014803]),
 'std_score_time': array([0.00507275, 0.01841115, 0.01388864, 0.01021053, 0.00435604,
        0.00252023, 0.00242891, 0.00092901, 0.00161789, 0.00383988]),
 'param_n_neighbors': masked_array(data=[3, 3, 5, 5, 10, 10, 12, 12, 15, 15],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value=999999),
 'param_weights': masked_array(data=['uniform', 'distance', 'uniform', 'distance',
                    'uniform', 'distance', 'uniform', 'distance',
            